In [1]:
import pandas as pd

In [2]:
eng = pd.read_csv("/content/eng.csv",sep = '/t',names = ['english'])

<ipython-input-2-7d4082158c8c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  eng = pd.read_csv("/content/eng.csv",sep = '/t',names = ['english'])


In [3]:
eng.head()

,english
0,"new jersey is sometimes quiet during autumn , ..."
1,the united states is usually chilly during jul...
2,"california is usually quiet during march , and..."
3,the united states is sometimes mild during jun...
4,"your least liked fruit is the grape , but my l..."


In [4]:
french = pd.read_csv('/content/french.csv', sep = '/t', names = ['french'])

<ipython-input-4-0ff2bd07b7f3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  french = pd.read_csv('/content/french.csv', sep = '/t', names = ['french'])


In [5]:
french.head()

,french
0,new jersey est parfois calme pendant l' automn...
1,les états-unis est généralement froid en juill...
2,"california est généralement calme en mars , et..."
3,"les états-unis est parfois légère en juin , et..."
4,"votre moins aimé fruit est le raisin , mais mo..."


# Data Preprocessing

In [6]:
import re

In [7]:
eng.info,french.info

(<bound method DataFrame.info of                                                   english
 0       new jersey is sometimes quiet during autumn , ...
 1       the united states is usually chilly during jul...
 2       california is usually quiet during march , and...
 3       the united states is sometimes mild during jun...
 4       your least liked fruit is the grape , but my l...
 ...                                                   ...
 137855  france is never busy during march , and it is ...
 137856  india is sometimes beautiful during spring , a...
 137857  india is never wet during summer , but it is s...
 137858  france is never chilly during january , but it...
 137859  the orange is her favorite fruit , but the ban...
 
 [137860 rows x 1 columns]>,
 <bound method DataFrame.info of                                                    french
 0       new jersey est parfois calme pendant l' automn...
 1       les états-unis est généralement froid en juill...
 2       california 

In [8]:
eng.isnull().sum(),french.isnull().sum()

(english    0
 dtype: int64,
 french    0
 dtype: int64)

In [9]:
def preprocess(text):
  text = re.sub('[!#?,.:";]'," ",text)
  return text

In [10]:
eng['english'] =eng['english'].apply(preprocess)
french['french'] =french['french'].apply(preprocess)

In [11]:
eng.head()

,english
0,new jersey is sometimes quiet during autumn ...
1,the united states is usually chilly during jul...
2,california is usually quiet during march and...
3,the united states is sometimes mild during jun...
4,your least liked fruit is the grape but my l...


In [12]:
english_words = []
french_words  = []

In [13]:
def get_unique(text,word_list):
  for word in text.split():
    if word not in word_list:
      word_list.append(word)

In [14]:
eng["english"].apply(lambda x:get_unique(x,english_words))
french["french"].apply(lambda x:get_unique(x,french_words))

0         None
1         None
2         None
3         None
4         None
          ... 
137855    None
137856    None
137857    None
137858    None
137859    None
Name: french, Length: 137860, dtype: object

In [15]:
eng_vocab_size=len(english_words)+1
french_vocab_size=len(french_words)+1

In [16]:
eng_vocab_size,french_vocab_size

(200, 351)

MAX Length of a sentence

In [17]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
max_length_eng = 0
for sentence in eng.english:
  tokens = nltk.word_tokenize(sentence)
  if(max_length_eng<len(tokens)):
    max_length_eng=len(tokens)
print("max length english",max_length_eng)


max length english 15


In [19]:
max_length_french = 0
for sentence in french.french:
  tokens = nltk.word_tokenize(sentence)
  if(max_length_french<len(tokens)):
    max_length_french=len(tokens)
print("max length english",max_length_french)

max length english 23


# Tokenization And Padding

In [20]:
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
def tokenize_and_pad(x,maxlen):
  tokenizer =Tokenizer(char_level=False)
  tokenizer.fit_on_texts(x)
  sequences = tokenizer.texts_to_sequences(x)
  padded = pad_sequences(sequences,maxlen=maxlen,padding='post')
  return tokenizer,sequences,padded

In [22]:
x_tokenizer, x_sequences, x_padded = tokenize_and_pad(eng.english, max_length_eng)
y_tokenizer, y_sequences, y_padded = tokenize_and_pad(french.french,  max_length_french)

In [23]:
x_sequences[0],x_padded[0]

([17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44],
 array([17, 23,  1,  8, 67,  4, 39,  7,  3,  1, 55,  2, 44,  0,  0],
       dtype=int32))

In [24]:
print("The Tokenized version of a sentence:\n",eng.english[-1:].item(),"\n is:\t",x_padded[-1:])

The Tokenized version of a sentence:
 the orange is her favorite fruit   but the banana is your favorite   
 is:	 [[ 5 84  1 32 11 13  6  5 87  1 29 11  0  0  0]]


In [25]:
print("The Tokenized version of a sentence:\n",french.french[-1:].item(),"\n is:\t",y_padded[-1:])

The Tokenized version of a sentence:
 l'orange est son fruit préféré   mais la banane est votre favori   
 is:	 [[84  1 20 16 17  5  7 87  1 40 93  0  0  0  0  0  0  0  0  0  0  0  0]]


# Split

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_padded, y_padded, test_size = 0.1)

# Model Building

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, TimeDistributed, RepeatVector, Embedding, Input, LSTM
from tensorflow.keras.models import Model

In [28]:
model = Sequential()
model.add(Embedding(eng_vocab_size,256,input_length=max_length_eng,mask_zero=True))
model.add(LSTM(256))#encoder
model.add(RepeatVector(max_length_french))#decoder repeatvector repeats the input for the desired number of times to change 2D-array to 3D array example: (1,256) to (1,23,256)
model.add(LSTM(256, return_sequences= True ))
model.add(TimeDistributed(Dense(french_vocab_size, activation ='softmax')))#repeats the layer for each time step so it will be applied independently to each of the n time steps in the input sequence.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 256)           51200     
                                                                 
 lstm (LSTM)                 (None, 256)               525312    
                                                                 
 repeat_vector (RepeatVector  (None, 23, 256)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 23, 256)           525312    
                                                                 
 time_distributed (TimeDistr  (None, 23, 351)          90207     
 ibuted)                                                         
                                                                 
Total params: 1,192,031
Trainable params: 1,192,031
Non-

In [29]:
# change the shape of target from 2D to 3D
import numpy as np
y_train = np.expand_dims(y_train, axis = 2)
y_train.shape

(124074, 23, 1)

In [30]:
model.fit(x_train, y_train, batch_size=1024, validation_split= 0.1, epochs=10)

Epoch 1/10
110/110 [==============================] - 442s 4s/step - loss: 2.7439 - accuracy: 0.4920 - val_loss: 2.2199 - val_accuracy: 0.5153
Epoch 2/10
110/110 [==============================] - 431s 4s/step - loss: 1.9279 - accuracy: 0.5637 - val_loss: 1.6377 - val_accuracy: 0.5994
Epoch 3/10
110/110 [==============================] - 425s 4s/step - loss: 1.5301 - accuracy: 0.6094 - val_loss: 1.4253 - val_accuracy: 0.6286
Epoch 4/10
110/110 [==============================] - 434s 4s/step - loss: 1.3337 - accuracy: 0.6459 - val_loss: 1.2572 - val_accuracy: 0.6585
Epoch 5/10
110/110 [==============================] - 432s 4s/step - loss: 1.1536 - accuracy: 0.6801 - val_loss: 1.0633 - val_accuracy: 0.7016
Epoch 6/10
110/110 [==============================] - 434s 4s/step - loss: 0.9952 - accuracy: 0.7171 - val_loss: 0.9497 - val_accuracy: 0.7290
Epoch 7/10
110/110 [==============================] - 431s 4s/step - loss: 0.8834 - accuracy: 0.7453 - val_loss: 0.8379 - val_accuracy: 0.7564

In [31]:
model.save("weights.h5")

# **Output**

In [32]:
def prediction(x, x_tokenizer = x_tokenizer, y_tokenizer = y_tokenizer):
    predictions = model.predict(x)[0]
    id_to_word = {id: word for word, id in y_tokenizer.word_index.items()}
    id_to_word[0] = ''
    return ' '.join([id_to_word[j] for j in np.argmax(predictions,1)])

In [34]:
def pad_to_text(padded, tokenizer):

    id_to_word = {id: word for word, id in tokenizer.word_index.items()}
    id_to_word[0] = ''

    return ' '.join([id_to_word[j] for j in padded])

In [35]:
for i in range(5):

  print('Original English word - {}\n'.format(pad_to_text(x_test[i], x_tokenizer)))
  print('Original French word - {}\n'.format(pad_to_text(y_test[i], y_tokenizer)))
  print('Predicted French word - {}\n\n\n\n'.format(prediction(x_test[i:i+1])))

Original English word - your favorite fruit is the grape but our favorite is the mango   

Original French word - votre fruit préféré est le raisin mais notre préféré est la mangue           

1/1 [==============================] - 2s 2s/step
Predicted French word - son fruit préféré est le pamplemousse mais son préféré est la la           




Original English word - new jersey is snowy during july and it is beautiful in winter   

Original French word - new jersey est la neige en juillet et il est beau en hiver          

1/1 [==============================] - 0s 34ms/step
Predicted French word - new jersey est jamais pluvieux en juillet et il est agréable en hiver          




Original English word - the united states is never freezing during february but it is sometimes rainy in fall

Original French word - les états unis ne sont jamais glaciales en février mais il est parfois pluvieux à l' automne      

1/1 [==============================] - 0s 34ms/step
Predicted French word - 